<a href="https://colab.research.google.com/github/GiovaniValdrighi/NOTEARS/blob/master/notears_nonlinear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import numpy as np
import scipy.optimize as sciops

In [0]:
class NOTEARS_MLP(tf.keras.models.Model):
  def __init__(self, n_variables, layers, bias = True):
    super(NOTEARS_MLP, self).__init__()
    self.n_variables = n_variables
    self.layers = layers
    self.fc1_pos = tf.keras.layers.Dense(n_variables * self.layers[0], input_shape = n_variables, use_bias = bias)
    self.fc1_neg = tf.keras.layers.Dense(n_variables * self.layers[0], inout_shape = n_variables, use_bias = bias)
    self.fc2 = tf.keras.Sequential()
    for l in layers:
      self.fc2.add(tf.keras.layers.LocallyConnected1D())

  def call(self, inputs):
    hid = self.fc1_pos(inputs) - self.fc1_neg(inputs)
    output = self.fc2(hid)
    return output

  def _bounds(self):
    #função para colocar os limites dos parâmetros
    pass

  def _h(self):
    '''Calculate the constraint of fc1 to ensure that it's a DAG'''
    fc1_weights = self.fc1_pos.weights - self.fc1_neg.weights
    tf.reshape(fc1_weights, [self.n_variables, self.n_variables])
    #preciso transformar na matrix de adjascencia
    #(Yu et al. 2019 DAG-GNN)
    # h(w) = tr[(I + kA*A)^n_variables] - n_variables
    M = tf.eye(n_variables, num_columns = n_variables) + A/n_variables
    E = tf.pow(M, n_variables - 1)
    h = tf.math.reduce_sum(tf.transpose(E) * M) - n_variables
  
  def _l2_loss(self):
    '''Calculate L2 loss from model parameters'''
    loss = 0
    fc1_weights = self.fc1_pos.weights - self.fc1_neg.weights
    loss +=  tf.math.reduce_sum(tf.pow(fc1_weights, 2))
    for layer in self.fc2:
      loss += tf.math.reduce_sum(tf.pow(layer.weights, 2))
    return loss

In [17]:
a**2

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 1,  4],
       [ 9, 16]], dtype=int32)>

In [0]:
max_iter = 1e16
rho = 1e20
c = 0.25
h_tol = 1e-4

rho = 0.
alpha = 0.
h = np.inf
for _ in range(max_iter):
  h_new = None
  W_new = None
  while rho < rho_max:
    optimizer.zero_grad()


    h_new = model.h_()
    if h_new > c * h:
      rho = 10*rho
    else:
      break
  alpha += rho * h_new
  if h <= h_tol or rho >= rho_max:
    break
W_est = model.fc1_to_adj()
W_est[np.abs(W_est) < threshold] = 0